In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
import re
from gensim.models import Word2Vec, Doc2Vec
from tensorflow.python.layers.core import Dense
from nltk.corpus import PlaintextCorpusReader
from nltk.tokenize import RegexpTokenizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import KFold

C:\Users\pc1\Anaconda3\envs\tensorflowenv\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
df = pd.read_csv('sentence_support_v2.tsv', delimiter='\t')
df.head(n=10)
sentence_support_df = pd.read_csv('sentence_support_v2.tsv', delimiter='\t')
sentence_support_df.drop_duplicates(inplace=True)
sentence_support_df.fillna(" ",inplace=True)
kf = KFold(n_splits=4, random_state=5, shuffle=True)

for train_index, test_index in kf.split(sentence_support_df):
    print("TRAIN:", train_index, "TEST:", test_index)
#     X_train, X_test = sentence_support_df.loc[train_index], sentence_support_df.loc[test_index]
sentence_support_df.head()

TRAIN: [    0     1     2 ..., 42319 42320 42321] TEST: [    3     5     9 ..., 42288 42308 42316]
TRAIN: [    0     2     3 ..., 42317 42318 42320] TEST: [    1     6     7 ..., 42315 42319 42321]
TRAIN: [    1     2     3 ..., 42319 42320 42321] TEST: [    0    11    15 ..., 42311 42314 42318]
TRAIN: [    0     1     3 ..., 42318 42319 42321] TEST: [    2     4    10 ..., 42312 42317 42320]


,pathwayA,pathwayB,crosstalk,pmid,sentenceFromPaper,label
0,Adipocytokine signaling pathway,ErbB signaling pathway,yes,23228483,"In the present study, we demonstrate that lept...",1
1,Adipocytokine signaling pathway,ErbB signaling pathway,yes,18945363,"In summary, our results suggest the existence ...",1
2,Adipocytokine signaling pathway,Estrogen signaling pathway,yes,20410173,These observations support the notion that the...,1
3,Adipocytokine signaling pathway,Estrogen signaling pathway,yes,23357303,The crosstalk between leptin and estrogen resc...,1
4,Adipocytokine signaling pathway,Estrogen signaling pathway,yes,22178935,The study supports the existence of a crosstal...,1


In [63]:
# for c in abstract_df.columns:
#     print(c)
#     print(abstract_df[c])
# # sents_df
abstract_df.fillna(' ', inplace= True)
abstract_df['all_words'] = abstract_df['Pathway A']+ ' ' + abstract_df['Pathway B'] + ' ' + abstract_df['Pubmed Query']+ ' ' + abstract_df['Molecule A'] + ' ' +  abstract_df['Molecule A Identifier']+ ' ' + abstract_df['Molecule A Source'] + ' ' + abstract_df['Molecule B']+ ' ' + abstract_df['Molecule B Identifier'] + ' ' + abstract_df['Molecule B Source']+ ' ' + abstract_df['Species'] + ' ' +  abstract_df['Tissue Name']+ ' ' + abstract_df['Condition'] + ' ' + abstract_df['Sentence from paper']+ ' ' + abstract_df['abstract'] 
abstract_df.head()

,Pathway A,Pathway B,Pubmed Query,PMID,Crosstalk,Transcriptional,Regulation type,Molecule A,Molecule A Identifier,Molecule A Source,...,BTO ID,Condition,Sentence from paper,Additional notes,Misleading sentences,abstract,class_label,text,sen_length,all_words
0,Adipocytokine signaling pathway,ErbB signaling pathway,Adipocytokine ErbB,25838314,no,no,,,,,...,,,,Mentions both pathways but does not say taht t...,,1. Poult Sci. 2015 Jun;94(6):1333-45. doi: 10...,0,1. Poult Sci. 2015 Jun;94(6):1333-45. doi: 10...,387,Adipocytokine signaling pathway ErbB signaling...
1,Adipocytokine signaling pathway,ErbB signaling pathway,Adipocytokine ErbB,23817051,no,no,,,,,...,,,,It is not mentioned that the pathways interact...,,1. Int J Obes (Lond). 2014 Mar;38(3):466-9. d...,0,1. Int J Obes (Lond). 2014 Mar;38(3):466-9. d...,339,Adipocytokine signaling pathway ErbB signaling...
2,Adipocytokine signaling pathway,ErbB signaling pathway,Leptin ErbB crosstalk,23228483,yes,yes,activating,LEP,P41159,UNIPROT,...,BTO:0000150,Cancer cells,"In the present study, we demonstrate that lept...",The paper does mention bidirectional crosstalk...,,1. Mol Oncol. 2013 Jun;7(3):379-91. doi: 10.1...,1,1. Mol Oncol. 2013 Jun;7(3):379-91. doi: 10.1...,350,Adipocytokine signaling pathway ErbB signaling...
3,Adipocytokine signaling pathway,ErbB signaling pathway,Leptin ErbB crosstalk,18945363,yes,unknown,unknown,LEP,P41159,UNIPROT,...,BTO:0000150,Cancer cells,"In summary, our results suggest the existence ...",It also says that it is important that leptin ...,,1. BMC Cancer. 2008 Oct 22;8:305. doi: 10.118...,1,1. BMC Cancer. 2008 Oct 22;8:305. doi: 10.118...,328,Adipocytokine signaling pathway ErbB signaling...
4,Adipocytokine signaling pathway,ErbB signaling pathway,Adpocytokine EGFR crosstalk,19047149,no,no,,,,,...,,,,The paper mentions crosstalk between leptin an...,,1. Cancer Res. 2008 Dec 1;68(23):9712-22. doi...,0,1. Cancer Res. 2008 Dec 1;68(23):9712-22. doi...,335,Adipocytokine signaling pathway ErbB signaling...


In [4]:
tokenizer = RegexpTokenizer(r'[^.?,:;\n\r\s]+')
sentences = df.sentenceFromPaper.unique()
sentences = [tokenizer.tokenize(s.lower()) for s in sentences]
seqlens = [len(s) for s in sentences]
df["sen_length"] = [len(x.split()) for x in df.sentenceFromPaper]
# df["pa_length"] = [len(x.split()) for x in df.pathwayA]
# df["pb_length"] = [len(x.split()) for x in df.pathwayB]
df.head()
df_fine = df[df.sen_length >0]
df_fine = df_fine[df_fine.sen_length <132]
splt = int(0.8*len(df_fine))
train_df = df_fine[:splt]
test_df = df_fine[splt:]
# df.at[20294,'sentenceFromPaper']
# df.at[4278,'sentenceFromPaper']
''' Discard sentences longer than 120/150 words'''
''' replace numbers with n-digit int, float or fraction'''
srtd = df_fine.sort_values(by= 'sen_length', ascending=True)
# srtd = df_fine.sort_values(by= 'pb_length', ascending=False)
srtd
# sentences[:5]

,pathwayA,pathwayB,crosstalk,pmid,sentenceFromPaper,label,sen_length
23337,Jak-STAT signaling pathway,Insulin signaling pathway,no,19319849,other direction,0,2
23538,GnRH signaling pathway,Neurotrophin signaling pathway,unclear,15948150,GnRH analogs,0,2
23325,Jak-STAT signaling pathway,Prolactin signaling pathway,no,20826756,other direction,0,2
277,Adipocytokine signaling pathway,TNF signaling pathway,yes,19183933,share receptor,1,2
23272,Jak-STAT signaling pathway,ErbB signaling pathway,no,19088723,other direction,0,2
290,HIF-1 signaling pathway,MAPK signaling pathway,yes,14978738,supports other direction,1,3
322,ErbB signaling pathway,Insulin signaling pathway,yes,19034632,supports other direction,1,3
137,ErbB signaling pathway,Notch signaling pathway,yes,23542173,HER2 inhibition activates Notch1,1,4
38163,NF-kappa B signaling pathway,Apoptosis,no,25967949,"pathways linked, but not crosstalk",0,5
136,ErbB signaling pathway,Notch signaling pathway,yes,23542173,"Interestingly, EGFR signaling inhibits Notch1 ...",1,7


In [3]:
tokenizer = RegexpTokenizer(r'[^.,?:;\n\r\s]+')
# abs_sentences = abstract_df.all_words.unique()
# abs_sentences = [tokenizer.tokenize(s.lower()) for s in abs_sentences]
# seqlens = [len(s) for s in sentences]
abstract_df["sen_length"] = [len(x.split()) for x in abstract_df.abstract]
# df["pa_length"] = [len(x.split()) for x in df.pathwayA]
# df["pb_length"] = [len(x.split()) for x in df.pathwayB]
abstract_df.head()
abstract_df_fine = abstract_df[abstract_df.sen_length >10]
abstract_df_fine.fillna('unknown', inplace=True)
abstract_df_fine['Pathways'] = abstract_df_fine['Pathway A']+ ' and ' + abstract_df_fine['Pathway B']
abstract_df_fine.loc[abstract_df_fine['Sentence from paper'] == 'unknown', 'Sentence from paper'] = 'No clear evidence'
abstract_df_fine.loc[abstract_df_fine.Crosstalk == 'no', 'Pathway A'] = 'unknown'
abstract_df_fine.loc[abstract_df_fine.Crosstalk == 'unclear', 'Pathway A'] = 'unknown'
abstract_df_fine.loc[abstract_df_fine.Crosstalk == 'no', 'Pathway B'] = 'unknown'
abstract_df_fine.loc[abstract_df_fine.Crosstalk == 'unclear', 'Pathway B'] = 'unknown'
abstract_df_fine.loc[abstract_df_fine.Crosstalk == 'no', 'Pathways'] = 'unknown'
abstract_df_fine.loc[abstract_df_fine.Crosstalk == 'unclear', 'Pathways'] = 'unknown'

# columns_of_interest = [-1,0,1,2,4,6,7,8,9,10,11,12,13,14,16,17,20]
columns_of_interest = [-1,0,1,2,4,6,7,8,9,10,11,12,13,14,16,20]
col_names = ['What are the pathways in the crosstalk',
                            'What is the first pathway in the crosstalk',
                            'What is the second pathway in the crosstalk',
                            'What is the Pubmed Query',
                            'Does the article support the crosstalk existence',
                            'What is the regulation type',
                            'What is the first Molecule',
                            'What is the first Molecule identifier',
                            'What is the first Molecule source',
                            'What is the second Molecule',
                            'What is the second Molecule identifier',
                            'What is the second Molecule source',
                            'What are the species',
                            'What is the tissue name',
                            'What is the condition',
#                             'What is the supporting or the denying sentence',
                            'abstract'
                           ]

abstract_df_fine = abstract_df_fine.take(columns_of_interest,axis=1)
abstract_df_fine.columns = col_names

abstract_df_fine.head()

# abstract_df_fine = abstract_df_fine[abstract_df_fine.sen_length <132]
splt = int(0.8*len(abstract_df_fine))
train_abstract_df = abstract_df_fine[:splt]
test_abstract_df = abstract_df_fine[splt:]
# # df.at[20294,'sentenceFromPaper']
# # df.at[4278,'sentenceFromPaper']
# ''' Discard sentences longer than 120/150 words'''
# ''' replace numbers with n-digit int, float or fraction'''
srtd = abstract_df.sort_values(by= 'sen_length', ascending=True)
# srtd = df_fine.sort_values(by= 'pb_length', ascending=False)
srtd
# len(srtd)
# # sentences[:5]

C:\Users\pc1\Anaconda3\envs\tensorflowenv\lib\site-packages\pandas\core\frame.py:3035: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)
C:\Users\pc1\Anaconda3\envs\tensorflowenv\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\pc1\Anaconda3\envs\tensorflowenv\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

,Pathway A,Pathway B,Pubmed Query,PMID,Crosstalk,Transcriptional,Regulation type,Molecule A,Molecule A Identifier,Molecule A Source,...,Tissue Name,BTO ID,Condition,Sentence from paper,Additional notes,Misleading sentences,abstract,class_label,text,sen_length
3113,ErbB signaling pathway,Adipocytokine signaling pathway,NaN,NO_RESULTS_FOR_PUBMED_QUERY,unclear,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ID list is empty! Possibly it has no correct I...,1,ID list is empty! Possibly it has no correct I...,10
3112,Apoptosis,Progesterone-mediated oocyte maturation,NaN,NO_RESULTS_FOR_PUBMED_QUERY,unclear,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ID list is empty! Possibly it has no correct I...,1,ID list is empty! Possibly it has no correct I...,10
3134,Prolactin signaling pathway,Hippo signaling pathway,NaN,NO_RESULTS_FOR_PUBMED_QUERY,unclear,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ID list is empty! Possibly it has no correct I...,1,ID list is empty! Possibly it has no correct I...,10
3114,GnRH signaling pathway,Adherens junction,NaN,NO_RESULTS_FOR_PUBMED_QUERY,unclear,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ID list is empty! Possibly it has no correct I...,1,ID list is empty! Possibly it has no correct I...,10
3115,GnRH signaling pathway,HIF-1 signaling pathway,NaN,NO_RESULTS_FOR_PUBMED_QUERY,unclear,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ID list is empty! Possibly it has no correct I...,1,ID list is empty! Possibly it has no correct I...,10
3116,GnRH signaling pathway,Hippo signaling pathway,NaN,NO_RESULTS_FOR_PUBMED_QUERY,unclear,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ID list is empty! Possibly it has no correct I...,1,ID list is empty! Possibly it has no correct I...,10
3117,HIF-1 signaling pathway,Prolactin signaling pathway,NaN,NO_RESULTS_FOR_PUBMED_QUERY,unclear,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ID list is empty! Possibly it has no correct I...,1,ID list is empty! Possibly it has no correct I...,10
3118,Hippo signaling pathway,Adipocytokine signaling pathway,NaN,NO_RESULTS_FOR_PUBMED_QUERY,unclear,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ID list is empty! Possibly it has no correct I...,1,ID list is empty! Possibly it has no correct I...,10
3119,Hippo signaling pathway,GnRH signaling pathway,NaN,NO_RESULTS_FOR_PUBMED_QUERY,unclear,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ID list is empty! Possibly it has no correct I...,1,ID list is empty! Possibly it has no correct I...,10
3120,Hippo signaling pathway,Prolactin signaling pathway,NaN,NO_RESULTS_FOR_PUBMED_QUERY,unclear,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ID list is empty! Possibly it has no correct I...,1,ID list is empty! Possibly it has no correct I...,10


In [4]:
# print(col_names[1:])
# new_df = abstract_df_fine.melt(id_vars=['abstract'], value_vars=col_names[:-1])
# new_df.head()

abs_train_df = train_abstract_df.melt(id_vars=['abstract'], value_vars=col_names[:-1])
abs_train_df.head()

abs_test_df = test_abstract_df.melt(id_vars=['abstract'], value_vars=col_names[:-1])
abs_test_df.tail()
# print(abs_test_df.loc[1,'variable'])
# len(abs_train_df)


,abstract,variable,value
10330,1. J Biol Chem. 2012 Feb 24;287(9):6490-502. ...,What is the condition,unknown
10331,1. PLoS One. 2013 Jul 29;8(7):e69994. doi: 10...,What is the condition,unknown
10332,1. Mol Vis. 2016 Feb 3;22:116-28. eCollection...,What is the condition,unknown
10333,1. Cancer Biol Ther. 2015;16(7):1099-109. doi...,What is the condition,unknown
10334,1. Tumour Biol. 2015 Mar;36(3):1613-8. doi: 1...,What is the condition,unknown


In [147]:
# ''' Run only once to generate the word2vec model'''

# corpus_root = "C:\\Users\pc1\\Downloads\\deep learning project\\pmcids"

# corpus = PlaintextCorpusReader(corpus_root, r'.*raw_text.txt')
# tokenizer2 = RegexpTokenizer(r'[^.?,:;\n\r]+')
# corpus.fileids()
# r = corpus.raw()

# r = re.sub(r'\d','d',r)

# all_sentences = [tokenizer.tokenize(sent.lower()) for sent in tokenizer2.tokenize(r)
#                      if len(sent)>1]

# all_sentences +=sentences
# all_sentences +=abs_sentences
# # len(all_sentences)
# word_model = Word2Vec(all_sentences, size=100, window=5, min_count=1, workers=4)
# word_model.save("w2v.mdl")


In [5]:
w2v_mdl = Word2Vec.load("w2v.mdl")
w2v_mdl.most_similar(positive=["results","suggest"], negative= ["study"])


[('indicate', 0.7018297910690308),
 ('indicating', 0.6676725149154663),
 ('suggesting', 0.6627146601676941),
 ('show', 0.6587152481079102),
 ('demonstrate', 0.651054859161377),
 ('demonstrating', 0.5896004438400269),
 ('suggests', 0.5773453712463379),
 ('suggested', 0.5762311816215515),
 ('showing', 0.5728246569633484),
 ('demonstrates', 0.5626481771469116)]

In [16]:
# tokenizer = RegexpTokenizer(r'[^.,:;\n\r\s]+')

def get_batch(start, batch_size, data_set):
    if start==0:
        data_set = data_set.sample(frac = 1)
    remainder = start + batch_size - len(data_set)
    if remainder > 0:
        return pd.concat([data_set[start:] , data_set[:remainder]]) , data_set
    else:
        return data_set[start:start+batch_size], data_set

def crossmap(out):
    if out=='yes': return 2
    elif out=='no': return 0
    else: return 1
    
def format_sent(sentence, max_length):
    n_rows = len(sentence)
    n_cols = len(sentence[0])
    if n_rows>=max_length:
        return sentence[:max_length]
    return np.concatenate((sentence,np.zeros([max_length-n_rows,n_cols])))

def att_enc_dec_inputs(batch, max_lengths):
    w2v_sents = [format_sent(w2v_mdl.wv[tokenizer.tokenize(str(s).lower())],max_lengths[0]) for s in batch.abstract]
    w2v_query = [format_sent(w2v_mdl.wv[tokenizer.tokenize(s.lower())],max_lengths[1]) for s in batch.variable]
    w2v_answer = [format_sent(w2v_mdl.wv[tokenizer.tokenize(s.lower())],max_lengths[2]) for s in batch.value]
#     start = [[w2v_mdl.wv['-']]]*len(batch)
#     w2v_answer_shifted = np.array([format_sent(np.concatenate((s,a)),max_lengths[2]) 
#               for s,a in zip(start,w2v_answer)])
#     return w2v_sents, w2v_query, w2v_answer_shifted, w2v_answer
    return w2v_sents, w2v_query, w2v_answer

def enc_dec_inputs(batch, max_lengths):
    w2v_sents = [format_sent(w2v_mdl.wv[tokenizer.tokenize(s.lower())],max_lengths[0]) for s in batch.sentenceFromPaper]
    w2v_pathA = [format_sent(w2v_mdl.wv[tokenizer.tokenize(s.lower())],max_lengths[1]) for s in batch.pathwayA]
    w2v_pathB = [format_sent(w2v_mdl.wv[tokenizer.tokenize(s.lower())],max_lengths[2]) for s in batch.pathwayB]
#     w2v_cross = [int(s) for s in batch.label]
    w2v_cross = [crossmap(s) for s in batch.crosstalk]
    return w2v_sents, w2v_pathA, w2v_pathB, w2v_cross

def single_rnn_inputs(batch, max_length):
    w2v_sents = [w2v_mdl.wv[tokenizer.tokenize(s.lower())] for s in batch.sentenceFromPaper]
    w2v_pathA = [w2v_mdl.wv[tokenizer.tokenize(s.lower())] for s in batch.pathwayA]
    w2v_pathB = [w2v_mdl.wv[tokenizer.tokenize(s.lower())] for s in batch.pathwayB]
    w2v_cross = [crossmap(s) for s in batch.crosstalk]
#     w2v_cross = [int(s) for s in batch.label]
    separator = [[w2v_mdl.wv['-']]]*len(batch)
    in_str = np.array([format_sent(np.concatenate((t,s,a,s,b)),max_length) 
              for t,s,a,b in zip(w2v_sents,separator,w2v_pathA,w2v_pathB)])
#     in_str = [np.concatenate((a,s,b,s,t)).tolist() for t,s,a,b in zip(w2v_sents,separator,w2v_pathA,w2v_pathB) ]
    return in_str, w2v_cross


In [ ]:

''' 
attentive reader model
Experiment Sen_QA_QB: Encoder input is the sentence, Decoder input is the question, Decoder produces the output as a Yes-No.
Loss function is still base on comparing word vectors
'''

max_sentence_length = 925
max_query_length = 10
max_output_length = 5
# num_layers = 2
truncate_length= 5
max_gradient_norm = 5
lrate = 0.001
num_neurons = 32 #128
embedding_length = 100
batch_size = 32 # 1
# num_classes = vocab_size

def nearest_word(vec):
    x = w2v_mdl.most_similar(positive=[vec], topn=1)
    return x[0][0]

graph_att = tf.Graph()
with graph_att.as_default():

    
    queries = tf.placeholder(tf.float32, [None, max_query_length, embedding_length])
    text = tf.placeholder(tf.float32, [None, max_sentence_length, embedding_length])
    decoder_outputs = tf.placeholder(tf.float32,[None, max_output_length, embedding_length],name='yhat')

    encoder_cell = tf.contrib.rnn.GRUCell(num_neurons)
    decoder_cell = tf.contrib.rnn.GRUCell(num_neurons)
    q_decoder_cell = tf.contrib.rnn.GRUCell(num_neurons)
    
#         encoder_cell = tf.contrib.rnn.BasicLSTMCell(num_neurons)
#         decoder_a_cell = tf.contrib.rnn.BasicLSTMCell(num_neurons)
#         decoder_b_cell = tf.contrib.rnn.BasicLSTMCell(num_neurons)

#     source_sequence_length = tf.transpose(encode_sent_lengths(text), [1, 0, 2])
    source_sequence_length = encode_sent_lengths(text)

    encoder_outputs, encoder_state = tf.nn.dynamic_rnn(encoder_cell,text,dtype=tf.float32,
                                         sequence_length = source_sequence_length)
    # attention_states: [batch_size, max_time, num_units]
    attention_states = tf.transpose(encoder_outputs, [1, 0, 2])

# Create an attention mechanism
#     attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(
    attention_mechanism = tf.contrib.seq2seq.LuongAttention(
        num_neurons, encoder_outputs, memory_sequence_length=source_sequence_length)
    q_decoder_cell = tf.contrib.seq2seq.AttentionWrapper(
        q_decoder_cell, attention_mechanism, attention_layer_size=num_neurons)

    q_dec_out, q_decoder_state = tf.nn.dynamic_rnn(q_decoder_cell,queries,dtype=tf.float32)
    
    projection_layer = Dense(embedding_length , use_bias=False) 
    
#     one_inputs = tf.ones(dtype=tf.float32,shape=[q_dec_out.shape[0], max_output_length, embedding_length])
#     zero_inputs = tf.zeros(dtype=tf.float32,shape=[q_dec_out.shape[0], max_output_length, embedding_length])
    sliced_out = q_dec_out[:,-max_output_length:,:]
    
    # Helper
    helper = tf.contrib.seq2seq.TrainingHelper(
        sliced_out, encode_sent_lengths(sliced_out))
#         one_inputs, encode_sent_lengths(one_inputs))
#         zero_inputs, encode_sent_lengths(zero_inputs))
    
#     attn_zero = q_decoder_cell.zero_state(batch_size=batch_size, dtype=tf.float32)
#     init_state = attn_zero.clone(cell_state=q_decoder_state).cell_state

    # Decoder
    decoder = tf.contrib.seq2seq.BasicDecoder(
        decoder_cell, helper, q_decoder_state.cell_state,
        output_layer=projection_layer)
    
    # Dynamic decoding
    outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder)
    yout = outputs.rnn_output
    
    mask = tf.cast(tf.greater_equal(decoder_outputs, 0),tf.float32)
#     crossent = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=decoder_outputs, logits=logits))
#     crossent = tf.nn.softmax_cross_entropy_with_logits(labels=decoder_outputs, logits=logits)
    mse = tf.losses.mean_squared_error(labels=decoder_outputs, predictions= yout)
    train_loss = (tf.reduce_sum(mse * mask) /  batch_size)
    
        # Helper
#     inf_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
#         nearest_word,
#         tf.fill([batch_size], 0), 1)

#     # Decoder
#     inf_decoder = tf.contrib.seq2seq.BasicDecoder(
#         decoder_cell, helper, q_decoder_state.cell_state,
#         output_layer=projection_layer)
#     # Dynamic decoding
#     inf_outputs, _ = tf.contrib.seq2seq.dynamic_decode(
#         decoder, maximum_iterations=max_output_length)

    
    # Optimization
    optimizer = tf.train.AdamOptimizer(lrate)
    params = tf.trainable_variables()
#     gradients = optimizer.compute_gradients(train_loss, params)
    gradients = tf.gradients(train_loss, params)
    clipped_gradients, _ = tf.clip_by_global_norm(gradients, max_gradient_norm)
    update_step = optimizer.apply_gradients(zip(clipped_gradients, params))
    

with tf.Session(graph=graph_att) as sess:
    init = tf.global_variables_initializer()
    init_l = tf.local_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    sess.run(init_l)

    av_acc = 0.0
    av_err = 0.0
    epochs = 6
    train_size = len(abs_train_df)
    
    for j in range(epochs):
        av_err = 0.0    
        i=0
        print("epoch " + str(j+1) + " :")
        while i< train_size:
            train_data, abs_train_df = get_batch(i, batch_size, abs_train_df )
            i += batch_size
            w2v_sents, w2v_query, w2v_answer = att_enc_dec_inputs(train_data,
                                                                        [max_sentence_length, max_query_length, max_output_length])
            _, ls, prediction = sess.run([update_step,train_loss,yout],
                                              {queries:w2v_query, 
                                               decoder_outputs:w2v_answer , text:w2v_sents })
            av_err += ls
        av_err /= (i/batch_size)
        
        print("epoch " + str(j+1) + ": finished epoch training with average loss = " 
                     + str(av_err))
       
        
    print()
    print("--------------------------Evaluation on Test Data---------------------------------------------------")
    print()
    
    test_data, abs_test_df = get_batch(0, batch_size, abs_test_df )
    w2v_sents, w2v_query, w2v_answer = att_enc_dec_inputs(test_data,
                                                                        [max_sentence_length, max_query_length, max_output_length])
    prediction = sess.run([yout],{queries:w2v_query, 
                                               decoder_outputs:w2v_answer , text:w2v_sents })
    preds = [[nearest_word(w) for w in s] for s in prediction[0]]
    
#     k=10
#     indeces = np.random.sample(range(), len(preds))
    for i in range(batch_size):
        print()
        print(test_data.loc[i,'variable'])
        print("answer: "+preds[i])
        print("as opposed to: "+ test_data.loc[i,'value'])
        print()
#     p,r,f,_ = precision_recall_fscore_support(w2v_cross, np.argmax(prediction[0],axis=1))
#     confusion_mat = confusion_matrix(w2v_cross, np.argmax(prediction[0],axis=1))
    
    # acc, prf, conf = evaluate(prediction,w2v_cross)
    # print("accuracy = " + str(acc))
#     print("Precision for the three classes (0=No, 1=Unk, 2=yes):")
#     print(p)
#     print("Recall for the three classes (0=No, 1=Unk, 2=yes):")
#     print(r)
#     print("F1-measure for the three classes (0=No, 1=Unk, 2=yes):")
#     print(f)

#     print("confusion matrix:")
#     print(confusion_mat)


In [34]:
def nearest_word(vec):
#     print(np.shape(vec))
    x = w2v_mdl.most_similar(positive=[vec], topn=1)
    return x[0][0]

# with tf.Session(graph=graph_att) as sess:     
#     print()
#     print("--------------------------Evaluation on Test Data---------------------------------------------------")
#     print()
    
#     test_data, abs_test_df = get_batch(0, batch_size, abs_test_df )
#     w2v_sents, w2v_query, w2v_answer = att_enc_dec_inputs(test_data,
#                                                                         [max_sentence_length, max_query_length, max_output_length])
#     prediction = sess.run([yout],{queries:w2v_query, 
#                                                decoder_outputs:w2v_answer , text:w2v_sents })
# print(np.shape(prediction))
preds = [[nearest_word(w) for w in s] for s in prediction[0]]
    
#     k=10
#     indeces = np.random.sample(range(), len(preds))
# print(list(test_data.variable)[i])
for i in range(batch_size):
    print()
    print(list(test_data.variable)[i])
    print("answer: "+ ' '.join(preds[i]))
    print("as opposed to: "+ list(test_data.value)[i])
    print()


What is the second Molecule source
answer: unknown no cancer and and
as opposed to: unknown


Does the article support the crosstalk existence
answer: no shh hedgehog hedgehog tgf-β
as opposed to: no


What is the tissue name
answer: malignant cell cell cell cell
as opposed to: unknown


What are the pathways in the crosstalk
answer: unknown cell and and and
as opposed to: HIF-1 signaling pathway and Neurotrophin signaling pathway


What is the second Molecule
answer: unknown no cellular cellular hsc
as opposed to: unknown


Does the article support the crosstalk existence
answer: no homo pathway cellular pathway
as opposed to: yes


What is the second pathway in the crosstalk
answer: unknown signaling pathway and and
as opposed to: unknown


What is the regulation type
answer: unknown cell emt emt hsc
as opposed to: unknown


What is the regulation type
answer: activating activating rddeydddddd transition malignant
as opposed to: unknown


What is the first Molecule source
answer: un